<a href="https://colab.research.google.com/github/JulieOnIsland/Machine_Learning/blob/main/kaggle_disaster_tweet_classification_LSTMs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Disaster Tweet Classification

```
Model 1: LSTM
Model 2: GRU
Model 3: BiLSTM
```

- You can download the data here:
https://www.kaggle.com/c/nlp-getting-started/data

In [ ]:
# Check GPU
!nvidia-smi -L

GPU 0: Tesla K80 (UUID: GPU-93aaf017-e172-fa85-e1ae-96212da4e78d)


# Load the data and look through the data


In [ ]:
import pandas as pd
path = '/content/drive/MyDrive/data/'
train_df = pd.read_csv(path + 'disaster_tweet_train.csv')
test_df = pd.read_csv(path + 'disaster_tweet_test.csv')
train_df.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [ ]:
test_df.head()

,id,keyword,location,text
0,0,NaN,NaN,Just happened a terrible car crash
1,2,NaN,NaN,"Heard about #earthquake is different cities, s..."
2,3,NaN,NaN,"there is a forest fire at spot pond, geese are..."
3,9,NaN,NaN,Apocalypse lighting. #Spokane #wildfires
4,11,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan


In [ ]:
len(train_df)

7613

In [ ]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7613 entries, 0 to 7612
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   id        7613 non-null   int64 
 1   keyword   7552 non-null   object
 2   location  5080 non-null   object
 3   text      7613 non-null   object
 4   target    7613 non-null   int64 
dtypes: int64(2), object(3)
memory usage: 297.5+ KB


- Would the keyword column matter when predicting the target? What is the probability of predicting target = 1 (disaster) when there's keyword given?

In [ ]:
keyword_not_nan = train_df.loc[train_df.keyword.isnull() == False]
keyword_not_nan

,id,keyword,location,text,target
31,48,ablaze,Birmingham,@bbcmtd Wholesale Markets ablaze http://t.co/l...,1
32,49,ablaze,Est. September 2012 - Bristol,We always try to bring the heavy. #metal #RT h...,0
33,50,ablaze,AFRICA,#AFRICANBAZE: Breaking news:Nigeria flag set a...,1
34,52,ablaze,"Philadelphia, PA",Crying out for more! Set me ablaze,0
35,53,ablaze,"London, UK",On plus side LOOK AT THE SKY LAST NIGHT IT WAS...,0
...,...,...,...,...,...
7578,10830,wrecked,NaN,@jt_ruff23 @cameronhacker and I wrecked you both,0
7579,10831,wrecked,"Vancouver, Canada",Three days off from work and they've pretty mu...,0
7580,10832,wrecked,London,#FX #forex #trading Cramer: Iger's 3 words tha...,0
7581,10833,wrecked,Lincoln,@engineshed Great atmosphere at the British Li...,0


In [ ]:
keyword_not_nan.keyword.unique()[:10]

array(['ablaze', 'accident', 'aftershock', 'airplane%20accident',
       'ambulance', 'annihilated', 'annihilation', 'apocalypse',
       'armageddon', 'army'], dtype=object)

In [ ]:
keyword_list = keyword_not_nan.keyword.unique()
keyword_list[5:10]

array(['annihilated', 'annihilation', 'apocalypse', 'armageddon', 'army'],
      dtype=object)

In [ ]:
len(keyword_list)

221

In [ ]:
keyword_not_nan.loc[keyword_not_nan.keyword == 'accident'].head()

,id,keyword,location,text,target
67,96,accident,CLVLND,'I can't have kids cuz I got in a bicycle acci...,0
68,97,accident,"Nashville, TN",Accident on I-24 W #NashvilleTraffic. Traffic ...,1
69,98,accident,"Santa Clara, CA",Accident center lane blocked in #SantaClara on...,1
70,100,accident,UK,http://t.co/GKYe6gjTk5 Had a #personalinjury a...,0
71,102,accident,"St. Louis, MO",#stlouis #caraccidentlawyer Speeding Among Top...,0


In [ ]:
len(keyword_not_nan.loc[keyword_not_nan.keyword == 'accident'])

35

In [ ]:
keyword_not_nan.loc[keyword_not_nan.keyword == 'accident'].target.value_counts()

1    24
0    11
Name: target, dtype: int64

In [ ]:
# aftershock
keyword_not_nan.loc[keyword_not_nan.keyword == keyword_list[2]].target.value_counts()

0    34
Name: target, dtype: int64

In [ ]:
# airplane%20accident
keyword_not_nan.loc[keyword_not_nan.keyword == keyword_list[3]].target.value_counts()

1    30
0     5
Name: target, dtype: int64

In [ ]:
if keyword_not_nan.loc[keyword_not_nan.keyword == keyword_list[2]].target.value_counts()[0]==34:
    key = keyword_not_nan.loc[keyword_not_nan.keyword == keyword_list[2]].target.value_counts()[1] = 0

key

0

In [ ]:
keyword_df = pd.DataFrame(data=keyword_list).rename(columns={0:'keyword'})
keyword_df.head()

,keyword
0,ablaze
1,accident
2,aftershock
3,airplane%20accident
4,ambulance


In [ ]:
keyword_df.iloc[0]

keyword    ablaze
Name: 0, dtype: object

In [ ]:
keyword_not_nan.loc[keyword_not_nan.keyword == keyword_list[0]].target.value_counts()

0    23
1    13
Name: target, dtype: int64

In [ ]:
keyword_not_nan.loc[keyword_not_nan.keyword == keyword_list[3]].target.value_counts()

1    30
0     5
Name: target, dtype: int64

In [ ]:
for i in range(len(keyword_list)):
    if len(keyword_not_nan.loc[keyword_not_nan.keyword == keyword_list[i]].target.value_counts()) == 1:
        a = keyword_not_nan.loc[keyword_not_nan.keyword == keyword_list[i]].target.value_counts()
        print(a)

0    34
Name: target, dtype: int64
1    37
Name: target, dtype: int64
1    39
Name: target, dtype: int64
1    39
Name: target, dtype: int64


In [ ]:
a_list = []
for i in range(len(keyword_list)):
    a = keyword_not_nan.loc[keyword_not_nan.keyword == keyword_list[i]].target.value_counts()
    a_list.append(a)
# print(a_list)
a_df = pd.DataFrame(a_list)
a_df
# a_df.loc[a_df[0].isnull() or a_df[1].isnull()]
a_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 221 entries, target to target
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   0       218 non-null    float64
 1   1       220 non-null    float64
dtypes: float64(2)
memory usage: 5.2+ KB


In [ ]:
len(a_df)

221

In [ ]:
a_df[0].fillna(0, inplace=True)
a_df[1].fillna(0, inplace=True)
a_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 221 entries, target to target
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   0       221 non-null    float64
 1   1       221 non-null    float64
dtypes: float64(2)
memory usage: 5.2+ KB


In [ ]:
a_df.head()
a_df['disaster_prob'] = a_df[1] / (a_df[1] + a_df[0])
a_df.head()

,0,1,disaster_prob
target,23.0,13.0,0.361111
target,11.0,24.0,0.685714
target,34.0,0.0,0.000000
target,5.0,30.0,0.857143
target,18.0,20.0,0.526316


In [ ]:
a_df.loc[a_df['disaster_prob'] > 0.5]

,0,1,disaster_prob
target,11.0,24.0,0.685714
target,5.0,30.0,0.857143
target,18.0,20.0,0.526316
target,10.0,22.0,0.687500
target,11.0,25.0,0.694444
...,...,...,...
target,9.0,18.0,0.666667
target,4.0,27.0,0.870968
target,4.0,29.0,0.878788
target,11.0,26.0,0.702703


In [ ]:
len(a_df.loc[a_df['disaster_prob'] > 0.5]) / len(a_df)

0.3891402714932127

- keyword 데이터가 있는 training data 중에서 봤을 때, disaster라고 예측한 비율이 50%가 넘는 데이터가 약 39% 정도가 된다. Thus, I will not use keyword column here.

# Model Experiment

In [ ]:
import tensorflow as tf
import tensorflow_hub as hub

In [ ]:
train_df['text'][0]

'Our Deeds are the Reason of this #earthquake May ALLAH Forgive us all'

In [ ]:
# Shuffle training dataframe
train_df_shuffled = train_df.sample(frac=1, random_state=88)
train_df_shuffled.head()

,id,keyword,location,text,target
4426,6299,hostage,NaN,quoted here--&gt;CNN: Purported ISIS video thr...,1
5579,7960,razed,NaN,The Latest: More Homes Razed by Northern Calif...,1
5830,8329,rubble,West Africa,#TNN: China's Stock Market Crash: Are There Ge...,1
4845,6900,mass%20murderer,"Haysville, KS",@CarlaChamorros HILLARY A MASS MURDERER.,0
7394,10581,windstorm,Chicago,'My Fifty Online Dates and why I'm still singl...,0


In [ ]:
train_df.target.value_counts() / len(train_df) * 100

0    57.034021
1    42.965979
Name: target, dtype: float64

In [ ]:
# split data into training and validation sets
from sklearn.model_selection import train_test_split
train_sentences, val_sentences, train_labels, val_labels = train_test_split(
    train_df_shuffled['text'].to_numpy(),
    train_df_shuffled['target'].to_numpy(),
    test_size = 0.1,
    random_state = 88
)

In [ ]:
len(train_sentences), len(train_labels), len(val_sentences), len(val_labels)

(6851, 6851, 762, 762)

In [ ]:
print(train_sentences[:10])
print(train_labels[:10])

['@JewhadiTM It is almost amazing to think someone thought suicide bombing would actually be a good idea.'
 'Greg Garza not in the 18 for Atlas tonight vs Leones Negros in Copa MX play. He left the previous game w/ an injury. #USMNT'
 'Max blew tf up ! ?????? shots fired ???? #CatfishMTV'
 "Officially skipping out on #FantasticFour/#Fant4stic/whatever the hashtag is. It's getting ANNIHILATED in reviews. Bummer."
 '@anarchic_teapot @BoironUSA @zeno001 Glononium 6C also helps with being blown up while bashing a bottle of nitroglycerin against a book.'
 'Listen to Landslide by Oh Wonder #SoundCloud https://t.co/SJkgJxff2r'
 'Wild land fires.. Here I come. ??????'
 'Inciweb OR Update:  Rogue River-Siskiyou National Forest Fires  8/5/15 12:00 PM (Rogue River-Siskiyou NF Area\x89Û_ http://t.co/LkwxU8QV7n'
 "STERLING-SCOTT on the Red Carpet at a fundraiser for 'OSO Mudslide' https://t.co/mA4ra7AtqL http://t.co/cg579wlDnE"
 "@jeannathomas not gonna lie.. I'm panicking a little bit. Vic/Hardy n

## Tokenization


In [ ]:
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization

In [ ]:
print(train_sentences[0].split(' '))
print(len(train_sentences[0].split(' ')))

['@JewhadiTM', 'It', 'is', 'almost', 'amazing', 'to', 'think', 'someone', 'thought', 'suicide', 'bombing', 'would', 'actually', 'be', 'a', 'good', 'idea.']
17


In [ ]:
avg_words = round(sum(len(train_sentences[i].split(' ')) for i in range(len(train_sentences))) / len(train_sentences))
avg_words

15

In [ ]:
# Set up text vectorizer variables
max_vocab_length = 10000         # total number of vocabs (most frequenct ones)
max_length = 15

In [ ]:
text_vectorizer = TextVectorization(max_tokens = max_vocab_length,
                                    output_mode = 'int',
                                    output_sequence_length = max_length)

In [ ]:
# Apply the text vectorizer to the training text
text_vectorizer.adapt(train_sentences)

In [ ]:
# Create a sample sentences and tokenize
sample_sent1 = 'I study Deep Learning because I want to improve the world by technology'
sample_sent2 = 'Warning: The avalanche is coming'
print(text_vectorizer([sample_sent1]))
print(text_vectorizer([sample_sent2]))

tf.Tensor(
[[   8 2281 1143 2073  157    8  137    5    1    2  103   17 1344    0
     0]], shape=(1, 15), dtype=int64)
tf.Tensor([[278   2 690   9 243   0   0   0   0   0   0   0   0   0   0]], shape=(1, 15), dtype=int64)


In [ ]:
# Choose random sentence from the training dataset and see how it tokenized
import random
random_sent = random.choice(train_sentences)
print(f'Original text:\n {random_sent}')
print(f'Tokenized Version:\n {text_vectorizer([random_sent])}')

Original text:
 The real question is why is the tornado siren going off in Dyersburg?
Tokenized Version:
 [[   2  384 1487    9   90    9    2  579  677  104   91    4    1    0
     0]]


In [ ]:
# What is the 5 most frequent and less frequent words?
words_in_vocab = text_vectorizer.get_vocabulary()
top_5_words = words_in_vocab[:5]
bottom_5_words = words_in_vocab[-5:]
print(f'Number of words in vocab: {len(words_in_vocab)}')
print(f'5 most common words: {top_5_words}')
print(f'5 least common words: {bottom_5_words}')

Number of words in vocab: 10000
5 most common words: ['', '[UNK]', 'the', 'a', 'in']
5 least common words: ['painful', 'paine', 'paging', 'pageshi', 'paeds']


## Embedding

In [ ]:
from tensorflow.keras import layers
embedding = layers.Embedding(input_dim = max_vocab_length,
                             output_dim = 128,
                             input_length = max_length)
embedding

In [ ]:
random_sentence = random.choice(train_sentences)
print(f'Original Text:\n{random_sentence}')
print(f'Tokenized Text:\n{text_vectorizer([random_sentence])}')
print(f'Embedded Text:\n{embedding(text_vectorizer([random_sentence]))}')
print(f'Embedded shape:\n{embedding(text_vectorizer([random_sentence])).shape}')

Original Text:
Wreckage 'Conclusively Confirmed' as From MH370: Malaysia PM: Investigators and the families of those who were... http://t.co/DtFSWNJZIL
Tokenized Text:
[[344 717 294  27  20 161 236 171 432   7   2 122   6 176  65]]
Embedded Text:
[[[-0.02231808  0.02484627  0.01348105 ...  0.02844118 -0.01467302
   -0.04393097]
  [-0.02661259 -0.03294507  0.02841412 ... -0.04232906  0.00548511
   -0.04274816]
  [ 0.02232033  0.00087061  0.01181749 ...  0.02887067  0.04035428
   -0.01419538]
  ...
  [-0.04162438  0.01637374  0.01549104 ...  0.03697428  0.00402658
    0.03405963]
  [ 0.02350657 -0.01058114 -0.01282167 ... -0.00985558 -0.0250322
    0.04325933]
  [ 0.02336632 -0.01645958  0.01131382 ...  0.01998888 -0.02115024
   -0.04804385]]]
Embedded shape:
(1, 15, 128)


## Confusion Matrix


In [ ]:
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

def calculate_results(y_true, y_pred):
    """
    Calculates model accuracy, precision, recall, and f1 score of a binary classification model
    """

    # Calculate model accuracy
    model_accuracy = accuracy_score(y_true, y_pred) * 100
    # Calculate model precision, recall, and f1-score using weighted average
    model_precision, model_recall, model_f1, _ = precision_recall_fscore_support(y_true, y_pred, average='weighted')
    model_results = {'accuracy': model_accuracy,
                     'precision': model_precision,
                     'recall': model_recall,
                     'f1': model_f1}
    return model_results

## Model1: LSTM

### Trial 1

In [ ]:
from tensorflow.keras import layers
inputs = layers.Input(shape=(1,), dtype='string')
x = text_vectorizer(inputs)
x = embedding(x)
x = layers.LSTM(128, activation='tanh')(x)
outputs = layers.Dense(1, activation='sigmoid')(x)
model_1 = tf.keras.Model(inputs, outputs, name='model_1_LSTM')

In [ ]:
model_1.summary()

Model: "model_1_LSTM"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization (TextVec  (None, 15)               0         
 torization)                                                     
                                                                 
 embedding (Embedding)       (None, 15, 128)           1280000   
                                                                 
 lstm (LSTM)                 (None, 128)               131584    
                                                                 
 dense (Dense)               (None, 1)                 129       
                                                                 
Total params: 1,411,713
Trainable params: 1,411,713
Non-trainable params: 0
____________________________________________

In [ ]:
# Compile the model
model_1.compile(loss='binary_crossentropy',
                optimizer='Adam',
                metrics=['accuracy'])

In [ ]:
# EarlyStopping callback
callback = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=3)

In [ ]:
# Fit the model
model_1_history = model_1.fit(train_sentences,
                              train_labels,
                              epochs=10,
                              validation_data = (val_sentences, val_labels),
                              callbacks=[callback],
                              verbose=2)

Epoch 1/10
215/215 - 14s - loss: 0.5092 - accuracy: 0.7543 - val_loss: 0.4058 - val_accuracy: 0.8255 - 14s/epoch - 67ms/step
Epoch 2/10
215/215 - 3s - loss: 0.3183 - accuracy: 0.8685 - val_loss: 0.4379 - val_accuracy: 0.8045 - 3s/epoch - 13ms/step
Epoch 3/10
215/215 - 3s - loss: 0.2214 - accuracy: 0.9171 - val_loss: 0.4641 - val_accuracy: 0.7913 - 3s/epoch - 14ms/step
Epoch 4/10
215/215 - 3s - loss: 0.1599 - accuracy: 0.9426 - val_loss: 0.6109 - val_accuracy: 0.7848 - 3s/epoch - 13ms/step
Epoch 5/10
215/215 - 3s - loss: 0.1161 - accuracy: 0.9575 - val_loss: 0.7224 - val_accuracy: 0.7769 - 3s/epoch - 14ms/step
Epoch 6/10
215/215 - 3s - loss: 0.0828 - accuracy: 0.9677 - val_loss: 1.0220 - val_accuracy: 0.7756 - 3s/epoch - 14ms/step
Epoch 7/10
215/215 - 3s - loss: 0.0750 - accuracy: 0.9679 - val_loss: 1.0599 - val_accuracy: 0.7441 - 3s/epoch - 13ms/step
Epoch 8/10
215/215 - 3s - loss: 0.0636 - accuracy: 0.9730 - val_loss: 0.9511 - val_accuracy: 0.7612 - 3s/epoch - 13ms/step
Epoch 9/10
215

In [ ]:
# Make predictions
model_1_pred_probs = model_1.predict(val_sentences)
model_1_pred_probs[:10]

# Squeeze model_1_2_pred_probs to compare apple to apple, orange to orange
model_1_preds = tf.squeeze(tf.round(model_1_pred_probs))

# Confusion Matrix
model_1_results = calculate_results(y_true=val_labels,
                                    y_pred=model_1_preds)
model_1_results

{'accuracy': 76.11548556430446,
 'f1': 0.7616467334270243,
 'precision': 0.7625172265156037,
 'recall': 0.7611548556430446}

### Trial 2

In [ ]:
from tensorflow.keras import layers
inputs = layers.Input(shape=(1,), dtype='string')
x = text_vectorizer(inputs)
x = embedding(x)
x = layers.LSTM(128, activation='tanh', return_sequences=True)(x)
x = layers.BatchNormalization()(x)
x = layers.LSTM(64, activation='tanh')(x)
outputs = layers.Dense(1, activation='sigmoid')(x)
model_1_2 = tf.keras.Model(inputs, outputs, name='model_1_2_LSTM')

In [ ]:
model_1_2.summary()

Model: "model_1_2_LSTM"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization (TextVec  (None, 15)               0         
 torization)                                                     
                                                                 
 embedding (Embedding)       (None, 15, 128)           1280000   
                                                                 
 lstm_1 (LSTM)               (None, 15, 128)           131584    
                                                                 
 batch_normalization (BatchN  (None, 15, 128)          512       
 ormalization)                                                   
                                                                 
 lstm_2 (LSTM)               (None, 64)             

In [ ]:
# Compile the model
model_1_2.compile(loss='binary_crossentropy',
                optimizer='Adam',
                metrics=['accuracy'])

In [ ]:
# Fit the model
model_1_2_history = model_1_2.fit(train_sentences,
                              train_labels,
                              epochs=10,
                              validation_data = (val_sentences, val_labels),
                              callbacks=[callback],
                              verbose=2)

Epoch 1/10
215/215 - 14s - loss: 0.1008 - accuracy: 0.9609 - val_loss: 0.5849 - val_accuracy: 0.7677 - 14s/epoch - 65ms/step
Epoch 2/10
215/215 - 3s - loss: 0.0686 - accuracy: 0.9715 - val_loss: 0.8175 - val_accuracy: 0.7664 - 3s/epoch - 15ms/step
Epoch 3/10
215/215 - 3s - loss: 0.0595 - accuracy: 0.9746 - val_loss: 0.8721 - val_accuracy: 0.7717 - 3s/epoch - 15ms/step
Epoch 4/10
215/215 - 3s - loss: 0.0844 - accuracy: 0.9676 - val_loss: 0.9407 - val_accuracy: 0.7520 - 3s/epoch - 15ms/step
Epoch 5/10
215/215 - 3s - loss: 0.0500 - accuracy: 0.9759 - val_loss: 0.9862 - val_accuracy: 0.7651 - 3s/epoch - 15ms/step
Epoch 6/10
215/215 - 3s - loss: 0.0482 - accuracy: 0.9772 - val_loss: 1.0468 - val_accuracy: 0.7782 - 3s/epoch - 15ms/step
Epoch 7/10
215/215 - 3s - loss: 0.0450 - accuracy: 0.9797 - val_loss: 1.0083 - val_accuracy: 0.7769 - 3s/epoch - 15ms/step
Epoch 8/10
215/215 - 3s - loss: 0.0428 - accuracy: 0.9794 - val_loss: 1.0946 - val_accuracy: 0.7612 - 3s/epoch - 15ms/step
Epoch 9/10
215

In [ ]:
# Make predictions
model_1_2_pred_probs = model_1_2.predict(val_sentences)
model_1_2_pred_probs[:10]

array([[1.3167521e-03],
       [8.7428518e-05],
       [1.6816637e-03],
       [9.9966824e-01],
       [5.7853349e-03],
       [7.3362463e-03],
       [7.0329587e-04],
       [2.4965833e-01],
       [5.3797143e-05],
       [9.9995494e-01]], dtype=float32)

In [ ]:
model_1_2_preds = tf.squeeze(tf.round(model_1_2_pred_probs))
print(f'Prediction:{model_1_2_preds[:10]}')
print(f'Ground Truth:{val_labels[:10]}')

Prediction:[0. 0. 0. 1. 0. 0. 0. 0. 0. 1.]
Ground Truth:[0 1 0 1 1 0 1 0 0 1]


In [ ]:
model_1_2_results = calculate_results(y_true=val_labels,
                                    y_pred=model_1_2_preds)
model_1_2_results

{'accuracy': 75.59055118110236,
 'f1': 0.7563332644697549,
 'precision': 0.7570222296156643,
 'recall': 0.7559055118110236}

### Trial 3

In [ ]:
from tensorflow.keras import layers
inputs = layers.Input(shape=(1,), dtype='string')
x = text_vectorizer(inputs)
x = embedding(x)
x = layers.LSTM(128, activation='tanh')(x)
x = layers.Dense(64, activation='relu')(x)
outputs = layers.Dense(1, activation='sigmoid')(x)
model_1_3 = tf.keras.Model(inputs, outputs, name='model_1_3_LSTM_dense')

In [ ]:
model_1_3.summary()

Model: "model_1_3_LSTM_dense"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization (TextVec  (None, 15)               0         
 torization)                                                     
                                                                 
 embedding (Embedding)       (None, 15, 128)           1280000   
                                                                 
 lstm_3 (LSTM)               (None, 128)               131584    
                                                                 
 dense_2 (Dense)             (None, 64)                8256      
                                                                 
 dense_3 (Dense)             (None, 1)                 65        
                                              

In [ ]:
# Compile the model
model_1_3.compile(loss='binary_crossentropy',
                optimizer='Adam',
                metrics=['accuracy'])

# Fit the model
model_1_3_history = model_1_3.fit(train_sentences,
                              train_labels,
                              epochs=10,
                              validation_data = (val_sentences, val_labels),
                              callbacks=[callback],
                              verbose=2)

Epoch 1/10
215/215 - 5s - loss: 0.1340 - accuracy: 0.9480 - val_loss: 0.8438 - val_accuracy: 0.7598 - 5s/epoch - 24ms/step
Epoch 2/10
215/215 - 2s - loss: 0.0504 - accuracy: 0.9785 - val_loss: 1.0260 - val_accuracy: 0.7572 - 2s/epoch - 10ms/step
Epoch 3/10
215/215 - 2s - loss: 0.0392 - accuracy: 0.9804 - val_loss: 1.3686 - val_accuracy: 0.7664 - 2s/epoch - 10ms/step
Epoch 4/10
215/215 - 2s - loss: 0.0454 - accuracy: 0.9791 - val_loss: 1.1628 - val_accuracy: 0.7467 - 2s/epoch - 10ms/step
Epoch 5/10
215/215 - 2s - loss: 0.0384 - accuracy: 0.9806 - val_loss: 1.4381 - val_accuracy: 0.7690 - 2s/epoch - 10ms/step
Epoch 6/10
215/215 - 2s - loss: 0.0344 - accuracy: 0.9839 - val_loss: 1.9557 - val_accuracy: 0.7638 - 2s/epoch - 10ms/step
Epoch 7/10
215/215 - 2s - loss: 0.0408 - accuracy: 0.9809 - val_loss: 1.8581 - val_accuracy: 0.7677 - 2s/epoch - 10ms/step
Epoch 8/10
215/215 - 3s - loss: 0.0403 - accuracy: 0.9818 - val_loss: 1.3380 - val_accuracy: 0.7612 - 3s/epoch - 12ms/step
Epoch 9/10
215/2

In [ ]:
# Make predictions
model_1_3_pred_probs = model_1_3.predict(val_sentences)
model_1_3_pred_probs[:10]

# Squeeze model_1_2_pred_probs to compare apple to apple, orange to orange
model_1_3_preds = tf.squeeze(tf.round(model_1_3_pred_probs))

# Confusion Matrix
model_1_3_results = calculate_results(y_true=val_labels,
                                    y_pred=model_1_3_preds)
model_1_3_results

{'accuracy': 74.54068241469817,
 'f1': 0.7460764687418157,
 'precision': 0.7474039429281867,
 'recall': 0.7454068241469817}

## Model2: GRU

In [ ]:
# Build model using GRU

from tensorflow.keras import layers
inputs = layers.Input(shape=(1,), dtype='string')
x = text_vectorizer(inputs)
x = embedding(x)
x = layers.GRU(128, activation='tanh')(x)
outputs = layers.Dense(1, activation='sigmoid')(x)
model_2 = tf.keras.Model(inputs, outputs, name='model_2_GRU')

In [ ]:
model_2.summary()

Model: "model_2_GRU"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_4 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization (TextVec  (None, 15)               0         
 torization)                                                     
                                                                 
 embedding (Embedding)       (None, 15, 128)           1280000   
                                                                 
 gru (GRU)                   (None, 128)               99072     
                                                                 
 dense_4 (Dense)             (None, 1)                 129       
                                                                 
Total params: 1,379,201
Trainable params: 1,379,201
Non-trainable params: 0
_____________________________________________

In [ ]:
# Compile the model
model_2.compile(loss='binary_crossentropy',
                optimizer='Adam',
                metrics=['accuracy'])

# Fit the model
model_2_history = model_2.fit(train_sentences,
                              train_labels,
                              epochs=10,
                              validation_data = (val_sentences, val_labels),
                              callbacks=[callback],
                              verbose=2)

Epoch 1/10
215/215 - 5s - loss: 0.1387 - accuracy: 0.9444 - val_loss: 0.7506 - val_accuracy: 0.7441 - 5s/epoch - 23ms/step
Epoch 2/10
215/215 - 2s - loss: 0.0491 - accuracy: 0.9775 - val_loss: 1.1993 - val_accuracy: 0.7454 - 2s/epoch - 9ms/step
Epoch 3/10
215/215 - 2s - loss: 0.0474 - accuracy: 0.9775 - val_loss: 1.4434 - val_accuracy: 0.7493 - 2s/epoch - 9ms/step
Epoch 4/10
215/215 - 2s - loss: 0.0365 - accuracy: 0.9820 - val_loss: 1.1168 - val_accuracy: 0.7205 - 2s/epoch - 9ms/step
Epoch 5/10
215/215 - 2s - loss: 0.0394 - accuracy: 0.9815 - val_loss: 1.6326 - val_accuracy: 0.7415 - 2s/epoch - 9ms/step
Epoch 6/10
215/215 - 2s - loss: 0.0425 - accuracy: 0.9800 - val_loss: 1.1988 - val_accuracy: 0.7572 - 2s/epoch - 9ms/step
Epoch 7/10
215/215 - 2s - loss: 0.0350 - accuracy: 0.9818 - val_loss: 1.4937 - val_accuracy: 0.7310 - 2s/epoch - 9ms/step
Epoch 8/10
215/215 - 2s - loss: 0.0323 - accuracy: 0.9832 - val_loss: 1.4596 - val_accuracy: 0.7533 - 2s/epoch - 9ms/step
Epoch 9/10
215/215 - 2s

In [ ]:
# Make predictions
model_2_pred_probs = model_2.predict(val_sentences)
model_2_pred_probs[:10]

model_2_preds = tf.squeeze(tf.round(model_2_pred_probs))

# Confusion Matrix
model_2_results = calculate_results(y_true=val_labels,
                                    y_pred=model_2_preds)
model_2_results

{'accuracy': 76.37795275590551,
 'f1': 0.7617139463688045,
 'precision': 0.7627827106718043,
 'recall': 0.7637795275590551}

## Compare model experiments

In [ ]:
model_1_results.values()

dict_values([76.11548556430446, 0.7625172265156037, 0.7611548556430446, 0.7616467334270243])

In [ ]:
import pandas as pd
model_1_results['accuracy'] = model_1_results['accuracy'] / 100
model_2_results['accuracy'] = model_2_results['accuracy'] / 100


results = {
    'model_1_LSTM' : list(model_1_results.values()),
    'model_2_GRU' : list(model_2_results.values())
}

df = pd.DataFrame(results, index=['accuracy', 'precision', 'recall', 'f1'])
df

,model_1_LSTM,model_2_GRU
accuracy,0.761155,0.763780
precision,0.762517,0.762783
recall,0.761155,0.763780
f1,0.761647,0.761714


- The results of LSTM and GRU are really similar!


## Model 3: Bidirectional

In [ ]:
model_3 = tf.keras.Sequential([
    text_vectorizer,
    embedding,
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64, return_sequences=True)),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32)),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(1)
])

# compilte the model
model_3.compile(loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
                optimizer='Adam',
                metrics=['accuracy'])



In [ ]:
model_3.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 text_vectorization (TextVec  (None, 15)               0         
 torization)                                                     
                                                                 
 embedding (Embedding)       (None, 15, 128)           1280000   
                                                                 
 bidirectional (Bidirectiona  (None, 15, 128)          98816     
 l)                                                              
                                                                 
 bidirectional_1 (Bidirectio  (None, 64)               41216     
 nal)                                                            
                                                                 
 dense_5 (Dense)             (None, 32)                2080      
                                                        

In [ ]:
# Fit the model
model_3_history = model_3.fit(train_sentences,
                              train_labels,
                              epochs=10,
                              validation_data = (val_sentences, val_labels),
                              callbacks=[callback],
                              verbose=2)

Epoch 1/10
215/215 - 14s - loss: 0.1042 - accuracy: 0.9526 - val_loss: 1.4434 - val_accuracy: 0.7388 - 14s/epoch - 65ms/step
Epoch 2/10
215/215 - 5s - loss: 0.0457 - accuracy: 0.9790 - val_loss: 1.5291 - val_accuracy: 0.7402 - 5s/epoch - 21ms/step
Epoch 3/10
215/215 - 5s - loss: 0.0387 - accuracy: 0.9804 - val_loss: 1.8197 - val_accuracy: 0.7467 - 5s/epoch - 21ms/step
Epoch 4/10
215/215 - 5s - loss: 0.0336 - accuracy: 0.9822 - val_loss: 2.3332 - val_accuracy: 0.7441 - 5s/epoch - 21ms/step
Epoch 5/10
215/215 - 4s - loss: 0.0305 - accuracy: 0.9828 - val_loss: 2.1249 - val_accuracy: 0.7625 - 4s/epoch - 21ms/step
Epoch 6/10
215/215 - 4s - loss: 0.0427 - accuracy: 0.9791 - val_loss: 1.8263 - val_accuracy: 0.7585 - 4s/epoch - 21ms/step
Epoch 7/10
215/215 - 4s - loss: 0.0342 - accuracy: 0.9820 - val_loss: 1.9500 - val_accuracy: 0.7493 - 4s/epoch - 21ms/step
Epoch 8/10
215/215 - 5s - loss: 0.0422 - accuracy: 0.9815 - val_loss: 1.5944 - val_accuracy: 0.7677 - 5s/epoch - 21ms/step


In [ ]:
# Make predictions
model_3_evaluate = model_3.evaluate(x=val_sentences, y=val_labels)
model_3_evaluate

24/24 [==============================] - 0s 11ms/step - loss: 1.5944 - accuracy: 0.7677


[1.5943875312805176, 0.7677165269851685]

# The END
